In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = None

In [106]:
df = pd.read_csv("../data/01-tiburon_4.csv", index_col = 0)
df.head(2)

year country  activity age     species_  mes deceso genero
0  2018     usa  Paddling  57  White shark  Jun      N      F
1  2018  brazil  Swimming  18  Tiger shark  Jun      Y      M

# Pair Programming Limpieza II
​
Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. 

**Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). 

El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 

💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

imaginemos que el valor de una celda es el siguiente

'White shark, 3.5 m'

tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 

 patrón que podriamos usar es: 

patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [107]:
df.head()

year    country        activity age          species_  mes deceso genero
0  2018        usa        Paddling  57       White shark  Jun      N      F
1  2018     brazil        Swimming  18       Tiger shark  Jun      Y      M
2  2018        usa         Walking  15    Bull shark, 6'  May      N      M
3  2018  australia  Feeding sharks  32   Grey reef shark  May      N      M
4  2018    england         Fishing  21  Invalid incident  May      N      M

In [108]:
def especies(x):
    patron_blanco = ".*[Ww](hite|HITE).*"
    patron_tiger = ".*[Tt](iger|IGER).*"
    patron_gris = ".*[Gg](rey|REY).*"
    patron_limon = ".*[lL](emon|EMON).*"
    patron_toro = ".*[bB](ull|ULL).*"
    try:
        if re.findall(patron_blanco, x):
            return "White"
        elif re.findall(patron_tiger, x):
            return "Tiger"
        elif re.findall(patron_gris, x):
            return "Grey"
        elif re.findall(patron_limon, x):
            return "Lemon"
        elif re.findall(patron_toro, x):
            return "Bull"
        else:
            return "Unspecified"
    except:
        return "Unspecified"

In [109]:
df["species_"] = df["species_"].apply(especies)

In [113]:
df["species_"].value_counts()

Unspecified    982
White          385
Tiger          157
Bull           104
Grey            30
Lemon           11
Name: species_, dtype: int64

In [114]:
df.head(2)

year country  activity age species_  mes deceso genero
0  2018     usa  Paddling  57    White  Jun      N      F
1  2018  brazil  Swimming  18    Tiger  Jun      Y      M

2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:

- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
- Puede que os salga un error similar a este:
TypeError: expected string or bytes-like object
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
df['nombre_columna'] = df['nombre_columna'].astype(str)


- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.

- Por último cambiad el tipo de la columna de string a integer.

In [97]:
df.dtypes

year         int64
country     object
activity    object
age         object
species_    object
mes         object
deceso      object
genero      object
especies    object
dtype: object

In [98]:
df['age'] = df['age'].astype(str)

In [99]:
def numeros(x):
    patron_digitos = "\d+"
    try:
        return re.findall(patron_digitos, x)[0]
    except:
        return np.nan

In [115]:
df["age"] = df["age"].apply(numeros)

In [116]:
df["age"] = df["age"].astype("Int64", errors= "raise")

In [103]:
df.edad.dtypes

Int64Dtype()

In [117]:
df.head()

year    country        activity  age     species_  mes deceso genero
0  2018        usa        Paddling   57        White  Jun      N      F
1  2018     brazil        Swimming   18        Tiger  Jun      Y      M
2  2018        usa         Walking   15         Bull  May      N      M
3  2018  australia  Feeding sharks   32         Grey  May      N      M
4  2018    england         Fishing   21  Unspecified  May      N      M

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [118]:
df.to_csv("../data/02-tiburon_4.csv")